In [1]:
import os
import logging
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch as th
from torch.utils.data import DataLoader, Dataset

from transformers import TrainingArguments, RobertaForSequenceClassification

from clearml import Task


from utils import prepare_grader_data, compute_metrics

In [2]:
# Define constants
SUDY_NUMBER = 1
data_used = "famous_and_semi"
EXPERIMENT_NAME = f'eval_study_{SUDY_NUMBER}_{data_used}'


# Set up environment
trained_models_path = f"./anon_grader/trained_models/"
data_dir = f"textwash_data/study{SUDY_NUMBER}/intruder_test/full_data_study.csv"
PRED_PATH = "./anon_grader/results/predictions_" + data_used + ".csv"
RESULTS_PATH = "./anon_grader/results/results_" + data_used + ".csv"

DEVICE = "cuda" if th.cuda.is_available() else "cpu"

logging.info(f'Working on device: {DEVICE}')

# Cancel wandb logging
os.environ["WANDB_DISABLED"] = "true"


# Set seeds
SEED = 42
np.random.seed(SEED)
th.manual_seed(SEED)



In [3]:
# Read the data
columns_to_read = ["type", "text", "file_id", "name", "got_name_truth_q2"]
raw_data = pd.read_csv(data_dir, usecols=columns_to_read)

# Aggregate by file_id and calculate the rate of re-identification
data = (
    raw_data.groupby(["type", "file_id", "name", "text"])
    .agg({"got_name_truth_q2": "mean"})
    .reset_index()
)
data.rename(columns={"got_name_truth_q2": "human_rate"}, inplace=True)

# Define population to use
data = data[data["type"].isin(["famous", "semifamous"])]

# Preprocess the data
test_dataset = prepare_grader_data(data, SEED, DEVICE)['test']

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
)

In [21]:
# Read the data
columns_to_read = ["type", "text", "file_id", "name", "got_name_truth_q2"]
raw_data = pd.read_csv(data_dir, usecols=columns_to_read)

# Aggregate by file_id and calculate the rate of re-identification
data = (
    raw_data.groupby(["type", "file_id", "name", "text"])
    .agg({"got_name_truth_q2": "mean"})
    .reset_index()
)
data.rename(columns={"got_name_truth_q2": "human_rate"}, inplace=True)

# Define population to use
data = data[data["type"].isin(["famous", "semifamous"])]

# Preprocess the data
test_dataset = prepare_grader_data(data, SEED, DEVICE)['test']

# Create a DataLoader for the test dataset
test_dataloader = iter(DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
))

In [22]:
batches = [batch for batch in test_dataloader]
batch = batches[0]

In [23]:
type(batch)

dict

In [ ]:
# Load model
model = RobertaForSequenceClassification.from_pretrained(